# `TEANAPS` APPSTORE REVIEW SCRAPPER
- 본 자료는 텍스트 마이닝을 활용한 연구 및 강의를 위한 목적으로 제작되었습니다.
- 본 자료를 강의 목적으로 활용하고자 하시는 경우 꼭 아래 메일주소로 연락주세요.
- 본 자료에 대한 허가되지 않은 배포를 금지합니다.
- 강의, 저작권, 출판, 특허, 공동저자에 관련해서는 문의 바랍니다.
- **Contact : ADMIN(admin@teanaps.com)**

---

> **\*\*\* 주의사항 \*\*\***  
본 자료에서 설명하는 웹크롤링하는 방법은 해당 기법에 대한 이해를 돕고자하는 교육과 이를 활용한 연구 목적으로 사용되었으며, 대량의 무단 크롤링 및 상업적 활용을 금합니다.

## 1. 데이터 수집 준비하기

### 1-1. Selenium 라이브러리 설치

In [1]:
# 가상의 브라우저를 컨트롤 할 수 있도록 도와주는 selenium 패키지를 설치합니다.
# 아래 주석을 해지하고 셀을 실행합니다.
# 설치는 한번만 수행하면 되며, 재설치시 Requirement already satisfied: ~ 라는 메시지가 출력됩니다.

#!pip install selenium

### 1-2. Chrome Driver 다운로드 및 경로설정

- Chrome Driver 다운로드 URL : http://chromedriver.chromium.org/downloads

In [2]:
# 본 Jupyter Notebook 파일과 동일한 경로에 Chrome Driver가 존재하는 경우 아래 경로를 그대로 사용합니다.

# Windows 운영체제
#DRIVER_PATH = "chromedriver.exe"

# MAC, Linux 운영체제
# - 경고메시지 출력 시 보안 및 개인정보보호 메뉴에서 "확인없이 허용" 클릭 필요함
DRIVER_PATH = "./chromedriver"

### 1-3. 라이브러리 Import 및 Chrome Driver 실행

In [3]:
# Python 코드를 통해 가상의 브라우저를 띄우기 위해 selenium 패키지를 import 합니다.
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

# selenium을 활용해 브라우저를 직접 띄우는 경우, 실제 웹서핑을 할때처럼 로딩시간이 필요합니다.
# 로딩시간 동안 대기하도록 코드를 구성하기위해 time 패키지를 import 합니다.
import time

# Python 코드를 통해 웹페이지에 정보를 요청하기 위해 BeautifulSoup, urllib 패키지를 import 합니다.
from bs4 import BeautifulSoup
import requests

In [4]:
# 브라우저에 임의로 User-agent 옵션을 넣어 Python 코드로 접속함을 숨깁니다.
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--user-agent="Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/66.0.3359.139 Safari/537.36"')
# Chrome Driver를 호출합니다.
driver = webdriver.Chrome(executable_path = DRIVER_PATH, options=chrome_options)

## 2. AppStore 리뷰 수집하기

---

### 2-1. 리뷰 URL 정보 입력

In [5]:
# 수집할 리뷰 페이지 URL 정보를 입력합니다.
URL = "https://apps.apple.com/kr/app/%EC%8B%A0%ED%95%9C%ED%8E%98%EC%9D%B4%ED%8C%90/id572462317#see-all/reviews"

# 스크롤을 내릴 횟수를 입력합니다.
# 스크롤을 여러번 내릴수록 더 많은 리뷰가 수집됩니다.
SCROLL_COUNT = 5

# 리뷰를 저장할 파일명
REVIEW_FILENAME = "appstore_review.txt"

### 2-2. 리뷰 스크롤 내리기

In [6]:
driver.get(URL)

for i in range(SCROLL_COUNT):
    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
    time.sleep(3)
    
review_box_list = driver.find_elements(By.CLASS_NAME, "we-customer-review")
print("리뷰개수 :", len(review_box_list))

리뷰개수 : 50


### 2-3. 리뷰 수집

In [7]:
f = open(REVIEW_FILENAME, "w", encoding = "utf-8")

progress = 0
for review_box in review_box_list:
    progress += 1
    print(progress, end="\r")
    
    # 리뷰별점 가져오기
    review_star = review_box.find_element(By.CLASS_NAME, "we-star-rating-stars")
    review_star_count = review_star.get_attribute("class")[-1]
    
    # 더보기 버튼 클릭하기: try-except 예외처리 적용
    try:
        review_box.find_element(By.CLASS_NAME, "we-truncate__button").click()
        # 리뷰 텍스트 가져오기
        review_text = driver.find_element(By.CLASS_NAME, "we-customer-review__body--modal").text
        review_text = review_text.replace("\n", " ").strip()
        # 팝업 레이어 닫기
        driver.find_element(By.CLASS_NAME, "we-modal__close").click()
    # 더보기 버튼이 없은 경우 Exception 처리하기
    except:
        review_text = review_box.find_element(By.CLASS_NAME, "we-clamp").text
        review_text = review_text.replace("\n", " ").strip()
        
    # 파일에 쓰기
    f.write(review_star_count + "\t" + review_text + "\n")
    print(review_star_count + "\t" + review_text + "\n")
f.close()

5	저는 항상 인터넷으로 구매할 때 정말 편리하게 사용하고 있어요!!

5	최근에 업데이트 된 것 같은데 앱을 켜고 택시 페이를 선택하면 다음 화면(결제 정보를 수신하고 있다...)으로 넘어가지 않아서 사용하지 않다가 요즘 잘 되네요. 속도도 빠르고 몇 번 했도 문제없이 결제까지 잘 되네요...저만 사용하는 건 아니죠? 임산부나 카드 내기 귀찮으신분들 워해 많이 홍보해서 이용하게 하면 좋을 것 같습니다!

5	어플 편리하게 잘 사용하고 있습니다.

5	신한페이판 마이리포트 기능 자주 사용하고 있어요~ 다른 은행이나 카드 어플 따로 설치할 필요가 없어서 좋아요ㅎㅎ 챗봇 파니 마스크 쓰고 있는 것도 너무 세심하고 귀엽습니다. 좋는 서비스 만들어주신 개발자님들 감사하구 힘내세요!!!!!!👍👍👍

5	결제가 너무 편해요. 신한카드만 쓰게 됩니다ㅠㅠ 진짜 이제까지 그렇게 많이 결제하면서 오류난 적 단 한번도 없어요. 서비스도 최고.. isp같은건 너무 혹평이 많아 무서워 쓸 수가 없는데 페이판은 그냥 프리패스로 결제가 되니 이것만한 앱 없는 것 같아요~~!

5	아이폰X 페이스ID가 활성화 되어 만족스럽게 사용중입니다. 애플워치4 GPS버전(WiFi only)에선 앱 자체가 로딩 안 되네요. (참고로 폰과 워치는 모두 정식 OS 최신버전 사용중입니다.) 감사합니다.

5	카드 쓰니까 앱을 안 쓸수는 없고 앱을 열어도 대체 12일 이후에 썼는 다음달에 청구될 돈이 얼만지 알수가 없어서 ... 얼마나 혜택을 받았는지 알려면 여러번 클릭해서 들어가봐야하고... 아주 불편했는데 이번 업그레이드 된후에는 이게 다 해결 됐네요! 업그레이드 아주 칭찬합니다 👏

5	혜택 어디서 어떻게 얼마 받았는지 꽁꽁 숨겨놓지 않고 찾기 쉬운곳에 상세하게 나와있다는 부분이 신뢰가 가네요. 속도도 다른 카드 앱보다 빠르고 꽤 직관적인듯... 평점이 왜 구린지 모르겠네;; 잘 쓰고 있습니다

5	타 뱅킹 많이써봤는데 UX부분에서 신한이 제일 편해요. 난 이제 이뱅킹앱 없으면 너무 불편하겠다..이런 

In [8]:
driver.close()